In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
data

<Response [200]>

In [3]:
#data.content

In [4]:
soup = BeautifulSoup(data.content)

In [5]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])     
    return articles

In [6]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)

In [7]:
#urllist

In [8]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

In [9]:
p_art =[]
for i in all_articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

In [11]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [12]:
cl_res.labels_

array([1, 3, 1, ..., 2, 2, 2])

In [13]:
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

NameError: name 'pd' is not defined

In [ ]:
Q.head()

In [ ]:
1 --> politics
2 --> religion
3 --> sports
4 --> entertainment
0 --> geopolitics

In [ ]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [ ]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

In [ ]:
Q